<a href="https://colab.research.google.com/github/BhaskarKumarSinha/Ml-Deep-Learning-AI-Projects/blob/main/DeepLearningProject/Code_Focused_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code-Focused Inference using GPT-2

## 🎯 Objective
The goal of this project is to load a **pre-trained GPT-2 model** and configure it so that it only answers **Python coding-related questions**.  
We will implement:
1. **Model Loading** – Load GPT-2 and tokenizer.  
2. **Filtering Mechanism** – Detect if the prompt is Python-related.  
3. **Response Generation** – Generate text if relevant.  
4. **Fallback Handling** – Reject non-coding prompts gracefully.  
5. **Testing** – Validate with different examples.  


### 1. Load Model & Tokenizer
We use Hugging Face’s `transformers` library:
- `AutoTokenizer` converts text into tokens.
- `AutoModelForCausalLM` loads GPT-2 for causal (left-to-right) language modeling.
- We wrap them in a `pipeline("text-generation")` for simplicity.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# -------------------------------
# 1. Load Model and Tokenizer
# -------------------------------
model_name = "gpt2"  # can also use "gpt2-medium" if GPU available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
# Create a text-generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


Device set to use cpu


### 2. Filtering Mechanism
Before answering, the system checks if the prompt is related to **Python coding**.  
We use **keyword-based filtering**:
- Example keywords: `"python"`, `"function"`, `"class"`, `"import"`, `"pandas"`, etc.
- If none of these appear, the model rejects the query.

In [3]:
# 2. Filtering Mechanism
# -------------------------------
def is_python_related(prompt: str) -> bool:
    """
    Simple keyword-based filter to check if the question is related to Python coding.
    """
    keywords = [
        "python", "code", "function", "class", "import",
        "list", "dictionary", "loop", "def", "print",
        "error", "exception", "numpy", "pandas", "tensorflow",
        "pytorch"
    ]
    prompt_lower = prompt.lower()
    return any(keyword in prompt_lower for keyword in keywords)

### 3. Response Generation
If the query is relevant, we let GPT-2 generate an answer:
- Use **sampling (`do_sample=True`)** with **top-p (nucleus sampling)** and **temperature** for diversity.
- Max length is capped at 200 tokens.

### 4. Handling Non-Coding Questions
If irrelevant, we return a **predefined fallback message**:  
> "I can only answer questions related to Python coding."

In [26]:
# 3 & 4. Generate or Reject Response

def generate_response(prompt: str) -> str:
    if is_python_related(prompt):
        # Add a coding bias to the prompt
        prefixed_prompt = "### Python Related code or explanation: \n" + prompt + "\n"
        outputs = text_generator(
            prefixed_prompt,
            max_new_tokens=250,   # only new tokens, avoids warning
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )
        return outputs[0]['generated_text']
    else:
        return "I can only answer questions related to Python coding."


### 5. Testing
We test with:
- Python-related prompts (✅ should answer).
- Non-coding prompts (❌ should reject).

In [27]:
# 5. Test
# -------------------------------
test_prompts = [
    "How do I write a function in Python to calculate factorial?",
    "What is the capital of France?",
    "Explain Python classes with an example.",
    "Tell me a joke.",
    "How can I import pandas in Python?"
]

In [28]:
for prompt in test_prompts:
    print(f"\nPrompt: {prompt}")
    print("Response:", generate_response(prompt))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Prompt: How do I write a function in Python to calculate factorial?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response: ### Python Related code or explanation: 
How do I write a function in Python to calculate factorial?
A simple example of this is:
import factorial def factorial ( x, y ): return x + y + factorial (x)
So, if you have two functions that are equivalent, you will need to calculate factorial(x, y) and calculate factorial(x, y).
If you want to read more about the syntax of factorial in Python, see my article: Factorial in Python.
I use this example in a couple of different ways. First, I use the factorial function in a function definition. The first example is a simple function that returns a function that takes an integer and returns an integer as the result. This example is more like a simple function that takes an integer and returns an integer as the result.
Second, I use the factorial function in a function definition. The first example is a simple function that takes an integer and returns an integer as the result. This example is more like a simple function that takes an int

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response: ### Python Related code or explanation: 
Explain Python classes with an example.

Use the module in your code.

Use the examples in your documentation.

Make sure you use the correct version of Python.

Use Python 2.7.

Use the latest version of Python.

Install the latest version of Python and try Python 3.3 or Python 3.4.

Use Python 3.4. Use Python 3.5.

Use Python 3.6.

Install the latest version of Python and try Python 3.7 or Python 3.8.

Use Python 3.8. Use Python 3.9.

Use Python 3.10.

Use Python 3.11.

Use Python 3.12.

Use Python 3.13.

Use Python 3.14.

Use Python 3.15.

Use Python 3.16.

Use Python 3.17.

Use Python 3.18.

Use Python 3.19.

Use Python 3.20.

Use Python 3.21.

Use Python 3.22.

Use Python 3.23.

Use Python 3.24.

Use Python 3.25.

Prompt: Tell me a joke.
Response: I can only answer questions related to Python coding.

Prompt: How can I import pandas in Python?
Response: ### Python Related code or explanation: 
How can I import pandas in Python?
Pi